In [2]:
# import relevant packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly as px
import plotly.graph_objs as go
import sys
sys.path.append("../src/")
sys.path.append("..")

import torch
from bert import *

In [3]:
# load bert
bert = BERTBase()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
# load bertspace -> feature space projection model
# buchanan model
# model = torch.load('../trained_models/model.plsr.buchanan.allbuthomonyms.5k.300components.500max_iters')

# binder model
model = torch.load('../trained_models/model.ffnn.binder.5k.50epochs.0.5dropout.lr1e-4.hsize300')

In [5]:
# read in data
df = pd.read_csv('../data/processed/varying_subject.csv')
df = df.drop(df.columns[0], axis = 1)
df.head()

,sentence,noun,n_score,verb,v_score,adjective,a_score,total score,construction,rating
0,It's nice of the person to do that,person,2,do,2,nice,2,6,nice-of-you,good
1,It's nice for the person to do that,person,2,do,2,nice,2,6,hard-for,good
2,It's nice that the person is doing that,person,2,do,2,nice,2,6,important-that,good
3,The person is nice to do that,person,2,do,2,nice,2,6,GAI,good
4,It's nice of the child to do that,child,2,do,2,nice,2,6,nice-of-you,good


In [6]:
# find feature labels
feature_map = model.feature_norms.feature_map
feature_labels = [str(feature_map.get_object(i)) for i in range(0, len(feature_map))]

feature_highlights = ['Face', 'Body', 'Pain', 'Audition', 'Speech', 'Head', 'Social', 'Human', 
                      'Communication', 'Self', 'Cognition', 'Benefit', 'Harm', 'Pleasant',
                      'Unpleasant', 'Happy', 'Sad', 'Angry', 'Disgusted', 'Fearful', 'Surprised',
                      'Drive', 'Needs', 'Attention', 'Arousal']

In [7]:
# define evaluation of each sentence/subject pair
def evaluate_nice_of_you(sentence, word):
      df = model.predict_top_n_features_in_context(word, sentence, bert = bert)
      df["sentence"] = sentence
      df = df.pivot(index = 'sentence', columns = 'Features', values = 'Values')
      return df 

# apply evaluations to each sentence
subj_evaluations_df = pd.concat([evaluate_nice_of_you(row['sentence'], row['noun']) for index, row in df.iterrows()])

# clean up df and add back into main df
new_df = df.merge(subj_evaluations_df, on = 'sentence')
new_df.reset_index(inplace = True)
new_df = new_df.drop(columns = ['index'])
new_df = new_df[~new_df.duplicated()]
df = new_df

In [8]:
# plot means
def plot_diffs(df, title):
    means = [df[col].mean() for col in df.columns]

    fig = go.Figure(layout = go.Layout(height = 1500, width = 500))

    colors = ['red' if col in feature_highlights else 'blue' for col in df.columns]

    fig.add_trace(go.Bar(x = means, y = df.columns, orientation = 'h', marker = dict(color = colors)))
    
    fig.update_layout(title = title)
    fig.show()


In [9]:
# determine movement of features from 'hard-for' to 'nice-of-you'
hard_for_df = df[(df["construction"] == "nice-of-you") | (df["construction"] == "hard-for")]


def subtract_hard_for(group):
    nice_of_you = group.loc[group['construction'] == 'nice-of-you', feature_labels].reset_index(drop = True)
    construction = group.loc[group['construction'] == 'hard-for', feature_labels].reset_index(drop = True)

    return nice_of_you.subtract(construction)


hard_for_diffs = hard_for_df.groupby("noun").apply(subtract_hard_for).reset_index(level = 1, drop = True)

plot_diffs(hard_for_diffs, "HARD FOR to NICE OF YOU")

In [10]:
# determine movement of features from 'GAI' to 'nice-of-you'
GAI_df = df[(df["construction"] == "nice-of-you") | (df["construction"] == "GAI")]


def subtract_GAI(group):
    nice_of_you = group.loc[group['construction'] == 'nice-of-you', feature_labels].reset_index(drop = True)
    construction = group.loc[group['construction'] == 'GAI', feature_labels].reset_index(drop = True)

    return nice_of_you.subtract(construction)


GAI_diffs = GAI_df.groupby("noun").apply(subtract_GAI).reset_index(level = 1, drop = True)

plot_diffs(GAI_diffs, "GAI to NICE OF YOU")

In [11]:
# determine movement of features from 'important-that' to 'nice-of-you'
important_that_df = df[(df["construction"] == "nice-of-you") | (df["construction"] == "important-that")]


def subtract_important_that(group):
    nice_of_you = group.loc[group['construction'] == 'nice-of-you', feature_labels].reset_index(drop = True)
    construction = group.loc[group['construction'] == 'important-that', feature_labels].reset_index(drop = True)

    return nice_of_you.subtract(construction)


important_that_diffs = important_that_df.groupby("noun").apply(subtract_important_that).reset_index(level = 1, drop = True)

plot_diffs(important_that_diffs, "IMPORTANT THAT to NICE OF YOU")